# OpenAI Usage

## Description 
Input and process sentences using OpenAI's API

In [1]:
import json
import os
import openai
from dotenv import load_dotenv
load_dotenv()  # take environment variables from .env

True

In [2]:
openai.api_key = os.environ.get("OPENAI_API_KEY")
sent_format = { 
    "sent_text": "string",
    "sent_person": "string",
    "sent_grade": "string",
    "sent_summary": "string",
    "sent_sentiment": "string",
    "sent_categories": [{
        "category": "int",
        "reason": "string"
    }]
}

chunk_format = {
    "verb_phrase": "string",
    "tense": "string",
    "verb_is_stative": "bool",
    "verb_voice": "string",
    "verb_sentiment": "string",
    "verb_categories": [{
        "category": "int",
        "reason": "string"
    }],
    "subjects": [{ 
        "subj_text": "string",
        "subj_is_plural": "boolean",
        "subj_semantics": "string",
        "semantic_category": "int"
    }],
    "objects": [{
        "obj_text": "string",
        "obj_is_plural": "boolean",
        "obj_semantics": "string",
        "semantic_category": "int"
    }],
    "errors": "boolean"
}


categories_prompt = 'The semantic categories are: ' \
    '1. acquisition such as by purchase or sale, gift-giving, donation, finding something, seizure, ' \
    'robbery and theft ' \
    '2. aggressive, hostile or criminal act such as attack, assault, coercion, intimidation, bribery, threatening, ' \
    'deception, dishonesty, betrayal, resistance, murder, homicide, invasion, occupation ' \
    '3. taking captive or capturing a person or animal ' \
    '4. agreement such as coming to an agreement, agreeing on something, and signing a contract ' \
    '5. disagreement or dispute ' \
    '6. violation of agreement ' \
    '7. any type of agricultural, apiculture, viniculture and aquacultural act or event ' \
    '8. any type of art and entertainment act or event including attending a movie, visiting a museum, ' \
    'playing a game, and hiking ' \
    '9. any attempt to do or achieve something ' \
    '10. avoidance such as bans, boycotts, escape, evasion, ignoring, overlooking, prevention and concealment ' \
    '11. bodily act such as movement, eating, drinking, grooming and sleeping ' \
    '12. change such as increase, decrease or physical change such as melting, bending and vaporization ' \
    '13. any type of cognition such as thinking, focusing, reading, assessing, characterizing, believing, making a ' \
    'decision, remembering or forgetting, having a goal, learning, having knowledge or skills, and understanding ' \
    'with the exception of emotions, feelings and sensory perception ' \
    '14. any type of emotion or feeling ' \
    '15. any type of sensory perception including pain and hunger ' \
    '16. any type of communication, communicating and speech acts such as recommending, acknowledging, denying, ' \
    'promising, making a request, boasting, joking, greeting, asking a question, granting permission, refusing, ' \
    'rejecting, deriding, slandering or surrendering ' \
    '17. competition or competing including playing a game ' \
    '18. continuation ' \
    '19. delay or wait' \
    '20. distribution or supply of something ' \
    '21. start of something ' \
    '22. end of something ' \
    '23. success ' \
    '24. failure ' \
    '25. win or loss ' \
    '26. any type of environmental or ecological event such as a disaster, weather event, or natural phenomenon ' \
    '27. any type of economic event such as a recesssion or inflation ' \
    '28. any type of financial event such as depositing or withdrawing money, releasing an annual report or ' \
    'paying taxes ' \
    '29. health-related event or act such as contracting a disease, addiction, a physical injury, frailty, ' \
    'having an allergic reaction, getting a vaccine, malnutrition and sterility ' \
    '30. impact and collision ' \
    '31. inclusion, unification and attachment such as adding to a list and assembling something ' \
    '32. separation ' \
    '33. information and data handling including IT operations ' \
    '34. issuing and publishing such as a newspaper, magazine or press release ' \
    '35. any legal or judicial event such as a trial, verdict, jury selection and judicial ruling ' \
    '36. life event such as birth, death, marriage and divorce ' \
    '37. measurement, measuring and reported assessment, count, percentages, etc. ' \
    '38. meeting and encounter such as party, chance encounter or ceremony ' \
    '39. any type of movement, travel or transportation such as entering or leaving a port or train station, ' \
    'traveling on a road, loading a truck or rail car, or making incremental changes such as pouring liquid from ' \
    'one container to another ' \
    '40. any political occurrence such as an election, coup or transfer of power, and political campaign ' \
    '41. any type of production, manufacture and creation event such as designing, building or producing ' \
    'products in a factory ' \
    '42. punishment ' \
    '43. reward and compensation ' \
    '44. removal of something ' \
    '45. restriction or blockage of movement, access, flow or freedom ' \
    '46. return, recovery and release ' \
    '47. risk taking including gambling ' \
    '48. search ' \
    '49. any type of space event such as a meteor shower, sun spots, eclipse, or rocket or satellite launch ' \
    '50. any type of storage event such as moving cargo into or from storage, stockpiling or hoarding ' \
    '51. substitution of something or someone for something or someone else ' \
    '52. utilization and consumption ' \
    '53. possession ' \
    '54. description of a characteristic or an attribute of a person, place, event, or thing such as its shape, ' \
    'temperature, color, weight, population, level of pollution, and percentage '

# Changed from 5 to 3 categories as 5 often was a 'stretch'
sent_prompt = 'You are a linguist and NLP expert. Article title: {article_title} ' \
    'Sentence text: {sent_text} ' + \
    f'Semantic categories: {categories_prompt} ' + \
    'For the Sentence provided, indicate if it is in the first, second or third person, its sentiment, ' \
    'and if there are grammatical or spelling errors in it. Also, estimate the expected school grade level ' \
    'where a reader understands the Sentence, and create a summary in 8 words or less. ' \
    'Indicate the numbers of the 3 closest semantic categories of the Sentence based on the context ' \
    'given by the title, and the words in the Sentence. Explain the reasoning for the categorizations. ' + \
    f'Return the response as a JSON object with a shape of {sent_format}'

chunk_prompt = 'You are a linguist and NLP expert. ' \
    'Here is a Chunk text ({chunk_text}) from the sentence ({sent_text}). ' + \
    f'Semantic categories: {categories_prompt} ' + \
    'Indicate the verb phrase of the Chunk, its tense, whether the verb is active or stative, and ' \
    'if the verb is in an active or passive voice. ' \
    'Make sure to capture the full text of phrasal verbs such as "lock up" or "talk down to", ' \
    'and the sentiment of the verb phrase - whether positive, negative or neutral. ' \
    'Indicate the numbers of the 3 closest semantic categories of the verb phrase based on the Sentence text, ' \
    'and the words in the Chunk. Explain the reasoning for the categorizations. ' \
    'Also return all the Chunk subjects and objects, if subjects and/or objects are present. ' \
    'Note that there may be 0, 1 or more than one subject or object. Find all of them. ' \
    'Indicate if any of the subjects and/or objects are singular or plural, and indicate their semantics as ' \
    '"person", "government", "organization, sub-organization or group of persons", "animal", ' \
    '"plant", "place", "environment or condition", "inanimate object or substance", "historical event", ' \
    '"law, policy or legal decision", "quantity or measurement", or "other". ' \
    'If possible, indicate the number of the closest semantic category of the subject or object. ' \
    'Explain the reasoning for the categorization. ' + \
    f'Return the response as a JSON object with a shape of {chunk_format}'

article_sent1 = "Rep. Liz Cheney (R-WY) compared herself to former President Abraham Lincoln during her concession speech shortly after her loss to Trump-backed Republican challenger Harriet Hageman."
article_sent2 = "Cheney claimed she lost her primary election only because she failed to “go along with President Trump’s lie about the 2020 election.”"
article_sent3 = "Trump urged GOP voters to reject his critic."
article_sent4 = 'U.S. Rep. Liz Cheney conceded defeat Tuesday in the Republican primary in Wyoming, an ' \
    'outcome that was a priority for former President Donald Trump as he urged GOP voters to reject ' \
    'one of his most prominent critics on Capitol Hill.'


In [4]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": 
         sent_prompt.replace("{article_title}", "Liz Cheney Loses WY Primary").replace("{sent_text}",article_sent1)}
    ],
    temperature=0.1
)

print(response)

{
  "id": "chatcmpl-7shKZFVsw5F5Jl9LSCPua8UTHWxCY",
  "object": "chat.completion",
  "created": 1693270631,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "{\n  \"sent_text\": \"Rep. Liz Cheney (R-WY) compared herself to former President Abraham Lincoln during her concession speech shortly after her loss to Trump-backed Republican challenger Harriet Hageman.\",\n  \"sent_person\": \"third\",\n  \"sent_grade\": \"12th\",\n  \"sent_summary\": \"Cheney compares herself to Lincoln in concession speech.\",\n  \"sent_sentiment\": \"neutral\",\n  \"sent_categories\": [\n    {\n      \"category\": 40,\n      \"reason\": \"The sentence mentions a political occurrence, specifically an election and a concession speech.\"\n    },\n    {\n      \"category\": 25,\n      \"reason\": \"The sentence mentions a loss in the election, indicating a win or loss event.\"\n    },\n    {\n      \"category\": 16,\n      

In [5]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": 
         sent_prompt.replace("{article_title}", "Liz Cheney Loses WY Primary").replace("{sent_text}",article_sent2)}
    ],
    temperature=0.1
)

print(response)

{
  "id": "chatcmpl-7shL9rHmhxees7GHFc2UMJb6z1Wn8",
  "object": "chat.completion",
  "created": 1693270667,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "{\n  \"sent_text\": \"Cheney claimed she lost her primary election only because she failed to \u201cgo along with President Trump\u2019s lie about the 2020 election.\u201d\",\n  \"sent_person\": \"third\",\n  \"sent_grade\": \"12th\",\n  \"sent_summary\": \"Liz Cheney lost primary for opposing Trump's election lie.\",\n  \"sent_sentiment\": \"negative\",\n  \"sent_categories\": [\n    {\n      \"category\": 40,\n      \"reason\": \"The sentence mentions a primary election, which falls under the political occurrence category.\"\n    },\n    {\n      \"category\": 16,\n      \"reason\": \"The sentence involves communication and speech acts, as it mentions Cheney's claim.\"\n    },\n    {\n      \"category\": 5,\n      \"reason\": \"The sentence

In [6]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": 
         sent_prompt.replace("{article_title}", "Liz Cheney Loses WY Primary").replace("{sent_text}",article_sent3)}
    ],
    temperature=0.1
)

print(response)

{
  "id": "chatcmpl-7shLLb1TLjNntSNxXogRgz93GlaYU",
  "object": "chat.completion",
  "created": 1693270679,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "{\n  \"sent_text\": \"Trump urged GOP voters to reject his critic.\",\n  \"sent_person\": \"third\",\n  \"sent_grade\": \"9th\",\n  \"sent_summary\": \"Trump urges rejection of GOP critic.\",\n  \"sent_sentiment\": \"negative\",\n  \"sent_categories\": [\n    {\n      \"category\": 40,\n      \"reason\": \"The sentence mentions a political occurrence, specifically an election campaign.\"\n    },\n    {\n      \"category\": 16,\n      \"reason\": \"The sentence involves communication and speech acts, as Trump is urging voters.\"\n    },\n    {\n      \"category\": 5,\n      \"reason\": \"There is a disagreement or dispute between Trump and his critic.\"\n    }\n  ]\n}"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_

In [7]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": 
         sent_prompt.replace("{article_title}", "Liz Cheney Loses WY Primary").replace("{sent_text}",article_sent4)}
    ],
    temperature=0.1
)

print(response)

{
  "id": "chatcmpl-7shLnrUfBIB9qr10TfMUZjAz1xQA9",
  "object": "chat.completion",
  "created": 1693270707,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "{\n  \"sent_text\": \"U.S. Rep. Liz Cheney conceded defeat Tuesday in the Republican primary in Wyoming, an outcome that was a priority for former President Donald Trump as he urged GOP voters to reject one of his most prominent critics on Capitol Hill.\",\n  \"sent_person\": \"third\",\n  \"sent_grade\": \"12th\",\n  \"sent_summary\": \"Liz Cheney loses Wyoming primary.\",\n  \"sent_sentiment\": \"negative\",\n  \"sent_categories\": [\n    {\n      \"category\": 40,\n      \"reason\": \"The sentence mentions an election and political campaign.\"\n    },\n    {\n      \"category\": 16,\n      \"reason\": \"The sentence involves communication acts, such as urging voters.\"\n    },\n    {\n      \"category\": 25,\n      \"reason\": \"The senten

In [8]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": 
         sent_prompt.replace("{article_title}", "").replace("{sent_text}", "The economy was sluggish.")}
    ],
    temperature=0.1
)

print(response)

{
  "id": "chatcmpl-7shLsdFlVS3kWsjf3AJamMRPBz32b",
  "object": "chat.completion",
  "created": 1693270712,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "{\n  \"sent_text\": \"The economy was sluggish.\",\n  \"sent_person\": \"third\",\n  \"sent_grade\": \"9th\",\n  \"sent_summary\": \"Sluggish economy.\",\n  \"sent_sentiment\": \"negative\",\n  \"sent_categories\": [\n    {\n      \"category\": 27,\n      \"reason\": \"The sentence mentions the economy, which is related to economic events.\"\n    },\n    {\n      \"category\": 12,\n      \"reason\": \"The word 'sluggish' implies a change in the economy.\"\n    },\n    {\n      \"category\": 37,\n      \"reason\": \"The sentence provides information about the state of the economy.\"\n    }\n  ]\n}"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 1062,
    "completion_tokens": 144,
    "total_tokens": 1206
  

In [10]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": 
         sent_prompt.replace("{article_title}", "").replace("{sent_text}", "The judge threw the book at the criminal.")}
    ],
    temperature=0.1
)

print(response)

{
  "id": "chatcmpl-7shOSeL1UmDRLumlHjlY6DegBrIBl",
  "object": "chat.completion",
  "created": 1693270872,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "{\n  \"sent_text\": \"The judge threw the book at the criminal.\",\n  \"sent_person\": \"third\",\n  \"sent_grade\": \"9th\",\n  \"sent_summary\": \"Judge punishes criminal severely.\",\n  \"sent_sentiment\": \"neutral\",\n  \"sent_categories\": [\n    {\n      \"category\": 2,\n      \"reason\": \"The sentence describes a criminal act.\"\n    },\n    {\n      \"category\": 35,\n      \"reason\": \"The sentence involves a legal event (trial and punishment).\"\n    },\n    {\n      \"category\": 42,\n      \"reason\": \"The sentence describes punishment.\"\n    }\n  ]\n}"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 1066,
    "completion_tokens": 135,
    "total_tokens": 1201
  }
}


In [11]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": 
         sent_prompt.replace("{article_title}", "").replace("{sent_text}", "Mary is not ugly.")}
    ],
    temperature=0.1
)

print(response)

{
  "id": "chatcmpl-7shOW4Ku6Gvhj8rIS7Mddw1RU8etB",
  "object": "chat.completion",
  "created": 1693270876,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "{\n  \"sent_text\": \"Mary is not ugly.\",\n  \"sent_person\": \"third\",\n  \"sent_grade\": \"6th\",\n  \"sent_summary\": \"Mary is not ugly.\",\n  \"sent_sentiment\": \"positive\",\n  \"sent_categories\": [\n    {\n      \"category\": 53,\n      \"reason\": \"The sentence describes a characteristic of Mary.\"\n    },\n    {\n      \"category\": 14,\n      \"reason\": \"The sentence does not express any emotion or feeling.\"\n    },\n    {\n      \"category\": 54,\n      \"reason\": \"The sentence describes the attribute of Mary's appearance.\"\n    }\n  ]\n}"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 1062,
    "completion_tokens": 134,
    "total_tokens": 1196
  }
}


In [12]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": 
         sent_prompt.replace("{article_title}", "").replace("{sent_text}", "Susan was apprehended by the police.")}
    ],
    temperature=0.1
)

print(response)

{
  "id": "chatcmpl-7shOaFPfdYZNwRED8HjfEnV4HxERE",
  "object": "chat.completion",
  "created": 1693270880,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "{\n  \"sent_text\": \"Susan was apprehended by the police.\",\n  \"sent_person\": \"third\",\n  \"sent_grade\": \"8th\",\n  \"sent_summary\": \"Susan arrested by police.\",\n  \"sent_sentiment\": \"neutral\",\n  \"sent_categories\": [\n    {\n      \"category\": 3,\n      \"reason\": \"The sentence describes the act of being captured by the police.\"\n    },\n    {\n      \"category\": 35,\n      \"reason\": \"The sentence involves a legal event, specifically the act of apprehension by the police.\"\n    },\n    {\n      \"category\": 36,\n      \"reason\": \"The sentence describes a life event, specifically the arrest of Susan.\"\n    }\n  ]\n}"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 1065,
    "co

In [16]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": 
         sent_prompt.replace("{article_title}", "")
         .replace("{sent_text}", "After she escaped from prison, Susan was apprehended by the police.")}
    ],
    temperature=0.1
)
print(response)
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": 
         chunk_prompt.replace("{chunk_text}", "Susan escaped from prison")
         .replace("{sent_text}", "After she escaped from prison, Susan was apprehended by the police.")}
    ],
    temperature=0.1
)
print(response)
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": 
         chunk_prompt.replace("{chunk_text}", "Susan was apprehended by the police")
         .replace("{sent_text}", "After she escaped from prison, Susan was apprehended by the police.")}
    ],
    temperature=0.1
)

print(response)

{
  "id": "chatcmpl-7shSLyUm50NkHq48TJMwCqXKEWx3S",
  "object": "chat.completion",
  "created": 1693271113,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "{\n  \"sent_text\": \"After she escaped from prison, Susan was apprehended by the police.\",\n  \"sent_person\": \"third\",\n  \"sent_grade\": \"8th\",\n  \"sent_summary\": \"Escapee Susan caught by police.\",\n  \"sent_sentiment\": \"neutral\",\n  \"sent_categories\": [\n    {\n      \"category\": 3,\n      \"reason\": \"The sentence describes the act of capturing a person.\"\n    },\n    {\n      \"category\": 2,\n      \"reason\": \"The sentence involves a criminal act of escaping from prison.\"\n    },\n    {\n      \"category\": 35,\n      \"reason\": \"The sentence mentions the involvement of the police, which relates to a legal event.\"\n    }\n  ]\n}"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 

In [17]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": 
         sent_prompt.replace("{article_title}", "").replace("{sent_text}", "Cheney's loss marked a remarkable fall of a powerful political dynasty.")}
    ],
    temperature=0.1
)

print(response)

{
  "id": "chatcmpl-7shSwRMmP2yEMuawhkAXLG0SBvL6w",
  "object": "chat.completion",
  "created": 1693271150,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "{\n  \"sent_text\": \"Cheney's loss marked a remarkable fall of a powerful political dynasty.\",\n  \"sent_person\": \"third\",\n  \"sent_grade\": \"12th\",\n  \"sent_summary\": \"Cheney's loss signifies a political dynasty's decline.\",\n  \"sent_sentiment\": \"negative\",\n  \"sent_categories\": [\n    {\n      \"category\": 40,\n      \"reason\": \"The sentence mentions a political occurrence, specifically the loss of Cheney.\"\n    },\n    {\n      \"category\": 23,\n      \"reason\": \"The sentence describes the failure of a powerful political dynasty.\"\n    },\n    {\n      \"category\": 25,\n      \"reason\": \"The sentence mentions the loss of Cheney, indicating a loss or defeat.\"\n    }\n  ]\n}"
      },
      "finish_reason": "sto